<a href="https://colab.research.google.com/github/esusrutha/zeroshot/blob/main/KnowledgeGraph1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910476 sha256=dae975c8c084bf4021e29485e4c25eb28347304cd2b14da9386b01c363b007aa
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [3]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=daab606b744ad1d30b08eef32482ac0bb31f715ce07c6c583ae9a310ae43e4e1
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
#Fixed code for S-BERT-KG embedding alignment
from sentence_transformers import SentenceTransformer
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
import pandas as pd

#Load the data from two csv files and merge them into a single dataframe
RESULTS_DIR = 'drive/MyDrive'
tweets_df1 = pd.read_csv(RESULTS_DIR + '/D1.csv', encoding='latin1', engine='python')
tweets_df2 = pd.read_csv(RESULTS_DIR + '/D22.csv', encoding='latin1', engine='python')
df = pd.concat([tweets_df1, tweets_df2], ignore_index=True)

#Load pre-trained S-BERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

#Define a function to embed a text using S-BERT
def embed_text(text):
  # Embed the text using S-BERT
  if isinstance(text, str):
      embeddings = model.encode(text)
  else:
    embeddings = model.encode([t for t in text if isinstance(t, str)])
    # Convert the embeddings to a PyTorch tensor
    tensor = torch.FloatTensor(embeddings)
  # Return the tensor
  return tensor

#Define a function to create a PyTorch Geometric graph from a knowledge graph
def create_kg_graph(kg):
  # Get the nodes and edges from the knowledge graph
  nodes = list(kg.keys())
  edges = []
  for node1 in kg.keys():
    for node2 in kg[node1]:
      if isinstance(node2, list):
        node2 = ', '.join(node2)
      if node1 in nodes and node2 in nodes:
        edges.append((nodes.index(node1), nodes.index(node2)))
        # Create the PyTorch Geometric graph data object
        data = Data(x=torch.eye(len(nodes)), edge_index=torch.LongTensor(edges).t())
  # Return the graph data object
  return data

#Define a knowledge graph
kg = {
'tweet': ['Advice','China','Mask','News','Transportation','USA','Vaccine'],
'Advice': ['Stay at home','wash hands','wear mask','social distancing'],
'China': ['Wuhan','China Coronavirus Updates','China news','other tweets related to China'],
'Mask': ['Mask shortage','wear mask','mask types','N50','N95','3M8210','3M9001','3M9322','3M9501'],
'News': ['Coronavirus updates','news','rules'],
'Transportation': ['Flights','traffic','traveling'],
'USA': ['U.S. Coronavirus Updates','COVID19','U.S. news','United States','US','USA'],
'Vaccine': ['Vaccine news','vaccine progress','vaccine injection'],
}
def create_kg_graph(kg, embeddings):
  # Get the nodes and edges from the knowledge graph
  nodes = list(kg.keys())
  edges = []
  for node1 in kg.keys():
    for node2 in kg[node1]:
      if isinstance(node2, list):
        node2 = ', '.join(node2)
      if node1 in nodes and node2 in nodes:
        edges.append((nodes.index(node1), nodes.index(node2)))
  # Create the PyTorch Geometric graph data object
  x = torch.FloatTensor(embeddings)
  data = Data(x=x, edge_index=torch.LongTensor(edges).t())
  return data
#Embed the text and create the PyTorch Geometric graph data object
text = df["hashtags"].apply(lambda x: ', '.join(x.strip("[]").split(", ")) if isinstance(x, str) else '')
embeddings = embed_text(text)
graph_data = create_kg_graph(kg, embeddings)


#Define a GCN model
class GCN(torch.nn.Module):
  def __init__(self, in_channels, out_channels):
    super(GCN, self).__init__()
    self.conv1 = GCNConv(in_channels=in_channels, out_channels=out_channels)
    self.conv2 = GCNConv(in_channels=out_channels, out_channels=out_channels)

  def forward(self, x, edge_index):
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.conv2(x, edge_index)
    return x
   
#Define a function  to train the GCN model
def train_gcn(model, data, embeddings, epochs):
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
  criterion = torch.nn.MSELoss()
  for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(data.x, data.edge_index)
    loss = criterion(output, embeddings)
    loss.backward()
    optimizer.step()
  print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

#Embed the text and create the PyTorch Geometric graph data object
text = df["hashtags"].apply(lambda x: ', '.join(x.strip("[]").split(", ")) if isinstance(x, str) else '')

embeddings = embed_text(text)

#Reshape the embeddings tensor to have shape (num_tweets, embedding_size)

embeddings = embeddings.reshape(len(text), embeddings.shape[1])

graph_data = create_kg_graph(kg, embeddings)

#Train the GCN model
model = GCN(embeddings.shape[1], embeddings.shape[1])
train_gcn(model, graph_data, embeddings, 100)

#Get the aligned embeddings
aligned_embeddings = model(graph_data.x, graph_data.edge_index)
print(aligned_embeddings)

Epoch 100/100, Loss: 0.1910123974084854
tensor([[ 0.1519, -0.2654,  0.7653,  ..., -0.0408,  0.0789, -0.0420],
        [ 0.1519, -0.2654,  0.7653,  ..., -0.0408,  0.0789, -0.0420],
        [ 0.1519, -0.2654,  0.7653,  ..., -0.0408,  0.0789, -0.0420],
        ...,
        [ 0.1519, -0.2654,  0.7653,  ..., -0.0408,  0.0789, -0.0420],
        [ 0.1519, -0.2654,  0.7653,  ..., -0.0408,  0.0789, -0.0420],
        [ 0.1519, -0.2654,  0.7653,  ..., -0.0408,  0.0789, -0.0420]],
       grad_fn=<AddBackward0>)


In [5]:
!pip install --upgrade tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install --upgrade transformers==4.28.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

#Load the pre-trained S-BERT model
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens", padding=True, truncation=True, max_length=128)
model = AutoModel.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")

#Load the data from two csv files and merge them into a single dataframe
RESULTS_DIR = 'drive/MyDrive'
tweets_df1 = pd.read_csv(RESULTS_DIR + '/D1.csv', encoding='latin1', engine='python')
tweets_df2 = pd.read_csv(RESULTS_DIR + '/D22.csv', encoding='latin1', engine='python')
df = pd.concat([tweets_df1, tweets_df2], ignore_index=True)
print(len(df))

#Read the string attributes of hashtags and convert them into lists
df["hashtags"] = df["hashtags"].apply(lambda x: x.strip("[]").split(", ") if isinstance(x, str) else [])

#Define the set of possible labels
labels = ["Vaccine", "USA", "Transportation", "News", "Mask", "China", "Advice"]

#Define the set of tweets to be categorized
tweets = df["hashtags"].tolist()

#Define the set of informative tweets for each label
advice = ["Stay at home", "wash hands", "wear mask", "social distancing"]
china = ["Wuhan", "China Coronavirus Updates", "China news", "other tweets related to China"]
mask = ["Mask shortage", "wear mask", "mask types", "N50", "N95", "3M8210", "3M9001", "3M9322", "3M9501"]
news = ["Coronavirus updates", "news", "rules"]
transportation = ["Flights", "traffic", "traveling"]
usa = ["U.S. Coronavirus Updates", "COVID19", "U.S. news", "United States", "US", "USA"]
vaccine = ["Vaccine news", "vaccine progress", "vaccine injection"]

#Define the set of informative sentences for each label
sentences = advice + china + mask + news + transportation + usa + vaccine

inputs = tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors="pt")
with torch.no_grad():
    model_output = model(**inputs)
sentence_embeddings = model_output.last_hidden_state.mean(dim=1)
print(sentence_embeddings[0])

#Embed the sentences and labels using the pre-trained S-BERT model
sentence_embeddings = model(tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")).last_hidden_state.mean(dim=1)
label_embeddings = model(tokenizer(labels, padding=True, truncation=True, return_tensors="pt")).last_hidden_state.mean(dim=1)

#Embed the tweets using the pre-trained S-BERT model
tweet_embeddings = model(tokenizer(tweets, padding=True, truncation=True, return_tensors="pt")).last_hidden_state.mean(dim=1)

#Construct a knowledge graph embedding space with the RE method
knowledge_graph = np.concatenate((sentence_embeddings.detach().numpy(), label_embeddings.detach().numpy()), axis=0)
projection_matrix = np.linalg.pinv(knowledge_graph.T @ knowledge_graph) @ knowledge_graph.T @ tweet_embeddings.detach().numpy()

#Classify the tweets based on their cosine similarity with the label embeddings
for i, tweet in enumerate(tweets):
  tweet_embedding = tweet_embeddings[i].detach().numpy()
  label_scores = cosine_similarity(tweet_embedding.reshape(1, -1) @ projection_matrix, label_embeddings.detach().numpy() @ projection_matrix)
  predicted_labels = [labels[j] for j in np.argsort(label_scores[0])[::-1][:3]]
  print(f"Tweet: {tweet}")

  print(f"Predicted labels: {predicted_labels}\n")

4018


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tensor([ 6.0923e-01,  1.7020e-01,  1.4263e+00,  5.5820e-02, -3.7053e-01,
        -7.9318e-01,  2.8696e-01,  4.9654e-02, -1.2787e-01, -2.2430e-01,
        -1.0987e+00,  6.3076e-02,  2.3494e-01,  2.6903e-01,  4.6296e-02,
        -1.6367e-02, -7.6356e-02, -3.6026e-02,  5.1514e-01, -4.1537e-02,
        -6.5193e-02,  4.8186e-02, -8.4061e-01, -5.9008e-01, -1.8842e-01,
        -4.2019e-01,  3.9145e-02,  6.3901e-01, -1.0088e+00,  3.6707e-01,
         3.2615e-01,  4.4237e-01,  7.8864e-01, -8.4532e-01,  1.9380e+00,
        -1.1278e-01,  7.3542e-02,  1.8683e-01, -4.6377e-02, -1.1875e+00,
         1.6106e-01, -1.6016e-01,  4.9172e-01,  5.1254e-01, -9.9598e-02,
        -2.8247e-01,  1.6135e+00,  1.9001e-01,  5.4326e-01, -9.9768e-01,
         9.3775e-01, -3.3511e-01,  2.1322e+00,  4.3280e-01, -2.0430e-01,
         5.1953e-01,  6.8108e-01, -4.3004e-01,  5.1755e-01,  6.1748e-01,
         1.1512e+00,  8.3632e-02,  3.9935e-01,  4.7074e-01, -1.0077e+00,
         4.1102e-01, -5.0340e-01, -1.9298e-01,  1.6

AttributeError: ignored

In [10]:
import pandas as pd
RESULTS_DIR = 'drive/MyDrive'
tweets_df1 = pd.read_csv(RESULTS_DIR + '/D1.csv', encoding='latin1', engine='python')
tweets_df2 = pd.read_csv(RESULTS_DIR + '/D22.csv', encoding='latin1', engine='python')
df = pd.concat([tweets_df1, tweets_df2], ignore_index=True)
print(len(df))


4018
